In [1]:
google_colab = False

# Google Colab用事前準備

In [2]:
# install
if google_colab:
  !pip install pybind11
  %cd /content
  !nvcc -V
  !sudo rm -fr BinaryBrain
  !rm -fr BinaryBrain
  !git clone --recursive -b ver3_develop https://github.com/ryuz/BinaryBrain.git
  %cd /content/BinaryBrain/python
  !sh copy_src.sh
  !python3 setup.py build
  !python3 setup.py develop

  import binarybrain as bb

In [3]:
import binarybrain as bb
import importlib
importlib.reload(bb)
importlib

<module 'importlib' from 'c:\\users\\ryuji\\appdata\\local\\programs\\python\\python37\\lib\\importlib\\__init__.py'>

In [4]:
# mount
if google_colab:
  from google.colab import drive 
  drive.mount('/content/drive')
  %cd /content/drive/My Drive/git-work/BinaryBrain_ver3_develop/tests/python

In [5]:
import binarybrain as bb
prop = bb.get_device_properties()
print(prop)


name                     : GeForce GT 1030
totalGlobalMem           : 2147483648
sharedMemPerBlock        : 49152
regsPerBlock             : 65536
warpSize                 : 32
memPitch                 : 2147483647
maxThreadsPerBlock       : 1024
maxThreadsDim[0]         : 1024
maxThreadsDim[1]         : 1024
maxThreadsDim[2]         : 64
maxGridSize[0]           : 2147483647
maxGridSize[1]           : 65535
maxGridSize[2]           : 65535
clockRate                : 1468000
totalConstMem            : 65536
major                    : 6
minor                    : 1
textureAlignment         : 512
deviceOverlap            : 1
multiProcessorCount      : 3
kernelExecTimeoutEnabled : 1
integrated               : 0
canMapHostMemory         : 1
computeMode              : 0




# メインコード

In [6]:
import binarybrain as bb
import numpy as np
import os
import sys
from tqdm import tqdm
from collections import OrderedDict

In [7]:
# load MNIST data
td = bb.load_cifar10()

## 元レイヤー

In [ ]:
modulation_size = 8
layer_rel2bin = bb.RealToBinary.create(modulation_size, framewise=True)
layer_bin2rel = bb.BinaryToReal.create(modulation_size)

# create layer
layer0_affine  = bb.DenseAffine.create([32])
layer0_norm    = bb.BatchNormalization.create()
layer0_act     = bb.Binarize.create()
layer1_affine  = bb.DenseAffine.create([32])
layer1_norm    = bb.BatchNormalization.create()
layer1_act     = bb.Binarize.create()
layer2_affine  = bb.DenseAffine.create([64])
layer2_norm    = bb.BatchNormalization.create()
layer2_act     = bb.Binarize.create()
layer3_affine  = bb.DenseAffine.create([64])
layer3_norm    = bb.BatchNormalization.create()
layer3_act     = bb.Binarize.create()
layer4_affine  = bb.DenseAffine.create([512])
layer4_norm    = bb.BatchNormalization.create()
layer4_act     = bb.Binarize.create()
layer5_affine  = bb.DenseAffine.create([10])
layer5_norm    = bb.BatchNormalization.create()
layer5_act     = bb.Binarize.create()
#layer5_act     = bb.ReLU.create()

# main network
cnv0_sub = bb.Sequential.create()
cnv0_sub.add(layer0_affine)
cnv0_sub.add(layer0_norm)
cnv0_sub.add(layer0_act)
layer0_cnv = bb.LoweringConvolution.create(cnv0_sub, 3, 3)

cnv1_sub = bb.Sequential.create()
cnv1_sub.add(layer1_affine)
cnv1_sub.add(layer1_norm)
cnv1_sub.add(layer1_act)
layer1_cnv = bb.LoweringConvolution.create(cnv1_sub, 3, 3)

cnv2_sub = bb.Sequential.create()
cnv2_sub.add(layer2_affine)
cnv2_sub.add(layer2_norm)
cnv2_sub.add(layer2_act)
layer2_cnv = bb.LoweringConvolution.create(cnv2_sub, 3, 3)

cnv3_sub = bb.Sequential.create()
cnv3_sub.add(layer3_affine)
cnv3_sub.add(layer3_norm)
cnv3_sub.add(layer3_act)
layer3_cnv = bb.LoweringConvolution.create(cnv3_sub, 3, 3)

main_net = bb.Sequential.create()

main_net.add(layer_rel2bin)
main_net.add(layer0_cnv)
main_net.add(layer1_cnv)
main_net.add(bb.MaxPooling.create(2, 2))
main_net.add(layer2_cnv)
main_net.add(layer3_cnv)
main_net.add(bb.MaxPooling.create(2, 2))
main_net.add(layer4_affine)
main_net.add(layer4_norm)
main_net.add(layer4_act)
main_net.add(layer5_affine)
main_net.add(layer5_norm)
main_net.add(layer5_act)
main_net.add(layer_bin2rel)

main_net.set_input_shape(td['x_shape'])

# Load
net_path = 'cifar10-dense-cnn'
os.makedirs(net_path, exist_ok=True)

if True:
    layer0_affine.load_json(os.path.join(net_path, 'layer0_affine.json'))
    layer1_affine.load_json(os.path.join(net_path, 'layer1_affine.json'))
    layer2_affine.load_json(os.path.join(net_path, 'layer2_affine.json'))
    layer3_affine.load_json(os.path.join(net_path, 'layer3_affine.json'))
    layer4_affine.load_json(os.path.join(net_path, 'layer4_affine.json'))
    layer5_affine.load_json(os.path.join(net_path, 'layer5_affine.json'))

    layer0_norm.load_json(os.path.join(net_path, 'layer0_norm.json'))
    layer1_norm.load_json(os.path.join(net_path, 'layer1_norm.json'))
    layer2_norm.load_json(os.path.join(net_path, 'layer2_norm.json'))
    layer3_norm.load_json(os.path.join(net_path, 'layer3_norm.json'))
    layer4_norm.load_json(os.path.join(net_path, 'layer4_norm.json'))
    layer5_norm.load_json(os.path.join(net_path, 'layer5_norm.json'))

main_net.send_command('parameter_lock false')
layer0_cnv.send_command('parameter_lock false')
layer1_cnv.send_command('parameter_lock false')

loss      = bb.LossSoftmaxCrossEntropy.create()
metrics   = bb.MetricsCategoricalAccuracy.create()
optimizer = bb.OptimizerAdam.create()
optimizer.set_variables(main_net.get_parameters(), main_net.get_gradients())

runner = bb.Runner(main_net, "cifar10-dense-cnn-binary", loss, metrics, optimizer)
runner.fitting(td, epoch_size=16, mini_batch_size=32, file_write=False, file_read=False, init_eval=True)

  0%|                                                      | 1/1563 [00:00<08:46,  2.97it/s, loss=1.84, accuracy=0.688]

[initial] test_accuracy=0.591500 test_loss=1.861303 train_accuracy=0.640520 train_loss=1.830796


epoch=1 test_accuracy=0.623200 test_loss=1.848601 train_accuracy=0.673900 train_loss=1.814997


 12%|██████                                              | 181/1563 [00:54<06:44,  3.42it/s, loss=1.84, accuracy=0.656]

In [ ]:
if True:
    layer0_affine.save_json(os.path.join(net_path, 'layer0_affine.json'))
    layer1_affine.save_json(os.path.join(net_path, 'layer1_affine.json'))
    layer2_affine.save_json(os.path.join(net_path, 'layer2_affine.json'))
    layer3_affine.save_json(os.path.join(net_path, 'layer3_affine.json'))
    layer4_affine.save_json(os.path.join(net_path, 'layer4_affine.json'))
    layer5_affine.save_json(os.path.join(net_path, 'layer5_affine.json'))

    layer0_norm.save_json(os.path.join(net_path, 'layer0_norm.json'))
    layer1_norm.save_json(os.path.join(net_path, 'layer1_norm.json'))
    layer2_norm.save_json(os.path.join(net_path, 'layer2_norm.json'))
    layer3_norm.save_json(os.path.join(net_path, 'layer3_norm.json'))
    layer4_norm.save_json(os.path.join(net_path, 'layer4_norm.json'))
    layer5_norm.save_json(os.path.join(net_path, 'layer5_norm.json'))